In [7]:
import os
import glob
import math
from PIL import Image, ImageOps
from sklearn.cluster import KMeans

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

In [8]:
img_rows, img_cols, img_chns = 92, 70, 3
show_name = "25_designers/anna-sui"

In [9]:
# Define PCA function
def pca(X):
    """
    Principal Component Analysis
    input: X, matrix with trainnig data stored as flattened arrays in rows
    return: projection matrix (with important dimensions first), variance and mean.

    SVD factorization:  A = U * Sigma * V.T
                        A.T * A = V * Sigma^2 * V.T  (V is eigenvectors of A.T*A)
                        A * A.T = U * Sigma^2 * U.T  (U is eigenvectors of A * A.T)
                        A.T * U = V * Sigma

    """

    # get matrix dimensions
    print("X shape", X.shape)
    num_data, dim = X.shape

    # center data
    mean_X = X.mean(axis=0)
    X = X - mean_X

    if dim > num_data:
        # PCA compact trick
        M = np.dot(X, X.T)  # covariance matrix
        e, U = np.linalg.eigh(M)  # calculate eigenvalues an deigenvectors
        tmp = np.dot(X.T, U).T
        V = tmp[::-1]  # reverse since the last eigenvectors are the ones we want
        S = np.sqrt(e)[::-1]  # reverse since the last eigenvalues are in increasing order
        for i in range(V.shape[1]):
            V[:, i] /= S
    else:
        # normal PCA, SVD method
        U, S, V = np.linalg.svd(X)
        V = V[:num_data]  # only makes sense to return the first num_data


    return U, V, S, mean_X

In [45]:
def run_pca_all(lwf_immatrix, show_name):

    tmp_img = np.array(Image.open(input_files[0]))
    m, n, k = tmp_img.shape

    print(lwf_immatrix[0].shape)

    bU, bV, bS, bimmean = pca(lwf_immatrix)

    n = 7
    figure = np.zeros((img_rows, img_cols * n, img_chns))

    show_name = show_name.replace("/", "_")
    file_path = "../result_img/%s/" % show_name
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    feature = 6
    cols = feature
    rows = 1
                                  
    avg_img = bimmean.reshape(img_rows, img_cols, img_chns)
    figure[0: img_rows, 0 : img_cols] = avg_img
    scipy.misc.imsave(file_path+"pca_avg.jpg", avg_img)

    for i in range(6):
        img = bV[i].reshape(img_rows, img_cols, img_chns)
        print(img)
        scipy.misc.imsave(file_path+"pca_%d.jpg" % (i+1), img)
        figure[0: img_rows,
               (i+1) * img_cols: (i + 1 + 1) * img_cols] = img

    scipy.misc.imsave(file_path+"pca_result.jpg", figure)

In [27]:
src_dir = "../img/%s" % show_name
#src_dir = "../img/chanel"
input_files = []
for root, dirs, files in os.walk(src_dir):
    for f in files:
        if os.path.splitext(f)[-1].lower() in [".jpg", ".JPG"]:
            os.path.join(root, f)
            input_files.append(os.path.join(root, f))

lwf_immatrix = []
for im in input_files:
    try:
        img = Image.open(im)
    except:
        pass
    if len(np.array(img).shape) != 3:
        print(im)
        plt.figure(figsize=(5, 5))
        plt.imshow(np.array(img))
        plt.show()
    else:
        #lwf_immatrix.append(np.array(img.resize((172, 260), Image.ANTIALIAS)))
        lwf_immatrix.append(np.array(img.resize((img_cols, img_rows), Image.ANTIALIAS)).flatten()) # change in img_row! in the above cell


lwf_immatrix = np.array(lwf_immatrix)
print(lwf_immatrix[0])

[ 48   9 167 ...,  81  78  90]


In [46]:
run_pca_all(lwf_immatrix, show_name)

(19320,)
X shape (964, 19320)


/sw/lsa/centos7/python-anaconda-arc-connect/created-20170421/lib/python3.5/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in sqrt


[[[ 0.00332431  0.00432452  0.00544243]
  [ 0.0034677   0.0045037   0.00573617]
  [ 0.0034662   0.00463526  0.00586535]
  ..., 
  [ 0.00652015  0.00672318  0.00763068]
  [ 0.00616133  0.00630952  0.00715041]
  [ 0.00655706  0.00660219  0.00744744]]

 [[ 0.00256003  0.00354977  0.00454239]
  [ 0.00254114  0.00362598  0.00467395]
  [ 0.00264101  0.00372976  0.0049282 ]
  ..., 
  [ 0.00640304  0.00653879  0.00734863]
  [ 0.00594848  0.00612391  0.0069211 ]
  [ 0.00645028  0.00651289  0.00735467]]

 [[ 0.00258069  0.00348448  0.00409392]
  [ 0.00265236  0.00366496  0.00428445]
  [ 0.00249213  0.0033569   0.00423101]
  ..., 
  [ 0.00678864  0.0067614   0.0073959 ]
  [ 0.00594643  0.00607648  0.00679191]
  [ 0.00608079  0.00624544  0.0070937 ]]

 ..., 
 [[ 0.00891157  0.00897213  0.00938828]
  [ 0.00891749  0.00896467  0.00937117]
  [ 0.00893421  0.00897473  0.0094097 ]
  ..., 
  [ 0.00889843  0.00898692  0.00933218]
  [ 0.00892463  0.00898531  0.00933294]
  [ 0.00892644  0.00899277  0.00934